<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [73]:
import tensorflow as tf
import edward as ed
import numpy as np

from edward.models import Beta, Exponential, Categorical, Multinomial

import networkx as nx

In [ ]:
# Number of nodes, max time and initial seed set (will update in future)
n=10
T = 100
seed = 0

# Store alpha as a set of beta distributions
alpha = Beta(tf.ones([10,10]),tf.ones([10,10]))

# Keep a list of infected nodes
infected = [seed]

# Initialize transmission times to be max time except for seed node
transmission = tf.ones(n)*T
transmission = tf.subtract(transmission,tf.one_hot(seed, n)*T)

# Keep track of time since last infected
latest_infected_time = 0.

# Iterate over all nodes
for i in range(n-1):

    # Calculate transmission times for the newest infected node
    latest_transmission = tf.add(latest_infected_time,Exponential(tf.gather_nd(alpha,[[infected[i]]])))
    
    # Calculate minimum times between this
    transmission = tf.minimum(transmission, latest_transmission)
    
    # Use to identify the lowest i nodes that have been infected. The latest on the list is the latest infected
    top_k = tf.nn.top_k(-transmission,k=i+2)
    
    # Store infected node and infected time    
    latest_infected = top_k.indices.eval()[0][i+1]
    latest_infected_time = -top_k.values.eval()[0][i+1]
    
    # Store infected node and infected time
    infected.append(latest_infected)
    
print(infected)
print(transmission.eval())

In [299]:
sess.run(transmission)

array([[0.        , 0.8429537 , 0.12118183, 0.08901425, 0.7668139 ,
        1.7763547 , 0.34023312, 0.694381  , 1.0494732 , 1.4189427 ]],
      dtype=float32)